# Agentic RAG
## Ограничения традиционного RAG

- Один шаг извлечения
- Несоответствия запроса и документов
- Ограниченный ризонинг - обычные RAG пайплайны не позволяют совершать многошаговый ризонинг для уточнения
- Ограничения по ширине контекстного окна - извлеченные документы должны подходить к контекстному окну модели

## Преимущества Agentic RAG 

- Формирование оптимизированных запросов: агент может преобразовать пользовательские запросы в запросы, подходящие для извлечения
- Множественное извлечение: агент может извлекать информацию итеративно, если нужно
- Ризонинг извлеченного контента: агент может анализировать, синтезировать и делать выводы
- Самокритика и уточнение: агент может оценить результаты извлечения и улучшить свой поход

## Данный подход естественным образом реализует продвинутые RAG техники: 
- HyDE (Hypothetical Document Embedding ): вместо использования пользовательского запроса напрямую, агент формулирует запросы, оптимальные для извлечения 
- Self-Query Refinement: агент может анализировать начальные результаты и производить последующие извлечения с уточненными запросами 

In [1]:
# !pip install datasets
# !pip install rank_bm25
# !pip install langchain-ollama
# !pip install langchain-core

# Построение Agentic RAG

In [1]:
import datasets
from langchain_core.documents.base import Document
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain.tools import BaseTool, tool
from pydantic import BaseModel, Field
from typing import Type
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent


/Users/karineayrapetyants/projects/agent-lectures/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Подготовка базы знаний 

- Будем использовать датасет, содержащий документацию HuggingFace
- Возьмем информацию, относящуюся к трансформерам

In [2]:
# Load the Hugging Face documentation dataset
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# Filter to include only Transformers documentation
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

# Convert dataset entries to Document objects with metadata
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

# Split documents into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Characters per chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],  # Priority order for splitting
)
docs_processed = text_splitter.split_documents(source_docs)

print(f"Knowledge base prepared with {len(docs_processed)} document chunks")

Knowledge base prepared with 14695 document chunks


## Tool для извлечения

In [3]:
class RetrieverInput(BaseModel):
    query: str = Field(..., description="The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.")
    
class RetrieverTool(BaseTool):
    args_schema: Type[BaseModel] = RetrieverInput
    return_direct: bool = True
    retriever: Type[BM25Retriever] = None

    name: str = "retriever"
    description: str = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        # Initialize the retriever with our processed documents
        self.retriever = BM25Retriever.from_documents(
            docs, k=10  # Return top 10 most relevant documents
        )

    def _run(self, query: str) -> str:

        """Execute the retrieval based on the provided query."""
        assert isinstance(query, str), "Your search query must be a string"

        # Retrieve relevant documents
        docs = self.retriever.invoke(query)

        
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

### Определим toolkit агента

In [4]:
retriever_tool = RetrieverTool(docs_processed)
toolkit = [retriever_tool]

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant.
    Use your tools to answer questions. If you do not have a tool to
    answer the question, say so. 
    Return only the answers.
    """),
    MessagesPlaceholder("chat_history", optional=True),
    MessagesPlaceholder("messages"),
])

## Агент

In [6]:
model = ChatOllama(model="gpt-oss:20b-cloud")
agent = create_react_agent(model, toolkit, prompt=prompt)

response = agent.invoke({
    "messages": [
        {"role": "user", "content": "For a transformers model training, which is slower, the forward or the backward pass?"}
    ]
})


/var/folders/zs/mv0lpz1n46q3ynw23qw42btc0000gn/T/ipykernel_92874/858769194.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model, toolkit, prompt=prompt)


In [7]:
for message in response["messages"]:
    print(message.pretty_print())
    print()

================================ Human Message =================================

For a transformers model training, which is slower, the forward or the backward pass?
None

================================== Ai Message ==================================
Tool Calls:
  retriever (881599fc-4212-40c4-87ed-5e7bc729cb66)
 Call ID: 881599fc-4212-40c4-87ed-5e7bc729cb66
  Args:
    query: transformers training forward pass slower than backward pass
None

================================= Tool Message =================================
Name: retriever


Retrieved documents:


===== Document 0 =====
Saving all activations from the forward pass in order to compute the gradients during the backward pass can result in 
significant memory overhead. The alternative approach of discarding the activations and recalculating them when needed 
during the backward pass, would introduce a considerable computational overhead and slow down the training process.

===== Document 1 =====
- A train step function w